In [1]:
using Pkg
using Random
Random.seed!(1234) ##settando a seed de números aleatórios
Pkg.add("DataFrames")
Pkg.add("Combinatorics")
using DataFrames
using Combinatorics
using Distributed

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


# Capítulo 1

# Exercício 8

Usando o método descrito no exercício anterior, escreva um programa para determinar uma solução inteira para a equação $ax + by = c$, tendo como entrada os coeficientes $a$, $b$ e $c$. A saída do programa deve ser, ou uma solução inteira da equação, ou uma mensagem indicando que tais soluções não existem. Portanto o programa consistirá, essencialmente, de uma implementação do algoritmo euclidiano extendido.

In [ ]:
function div(a,b) #sem usar a divisão e resto implementada pelo computador
    if b == 0
        throw(DivideError)
    elseif a == 0
        return 0
    end
    n = 1
    while n * b < a
        n = n * 2
    end
    while n * b > a
        n -= 1
    end
    return n
end

In [ ]:
div(1234,1234)

In [ ]:
div(1,12)

In [ ]:
div(10,0)

In [ ]:
div(110,109)

In [ ]:
function mdc(a,b)
    if a == 1
        return 1
    elseif b == 1
        return 1
    elseif a == 0
        return b
    elseif b == 0
        return a
    else
        A = max(a,b)
        B = min(a,b)
        Q = div(A,B) #quis implementar um algoritmo de divisão que não fosse preguiçoso
        if Q == 0
            return B
        else
            return mdc(B,A - B * Q)
        end
    end
end

In [ ]:
@time mdc(1234,54)

In [ ]:
function eext_aux(r1,r2,x1, x2, y1, y2)
    #queremos achar q e r tais que r= r2 + r1 q onde 0<=rj<r1
    qj = div(r2,r1)
    rj = r2 - r1 * qj
    if rj == 0
        return [x1,y1]
    else
        begin
        xj = x2 - qj * x1
        yj = y2 - qj * y1
        return eext_aux(rj,r1,xj,x1,yj,y1)
        end
    end
end

In [ ]:
function euclidiano_extendido(a,b,c)
    if c % mdc(a,b) != 0
        return "Não tem solução"
    elseif a > b
        return eext_aux(a,b,0,1,1,0)
    else
        return eext_aux(b,a,0,1,1,0)
    end
end

In [ ]:
@time euclidiano_extendido(1234,54,2)

## Exercício 9

Escreva um programa que, tendo como entrada dois inteiro $a$ e $b$, determina o máximo divisor comum de $a$ e $b$. Adapte seu programa para gerar aleatoriamente pares de inteiros $a$ e $b$ e calcular o $\mbox{mdc}(a,b)$. O programa deve ter como entrada o número total de pares que você deseja testar e, como saída o quociente $$\frac{\mbox{total de pares cujo mdc é }1}{\mbox{total de pares testados}}.$$

Este quociente dá uma medida da probabilidae de que um par de inteiros escolhido aleatoriamente seja co-primo. Vai ser necessário deixar o programa testar um número muito grande de pares para obter uma boa aproximação da probabilidade. Faça uma tabela com esses valores, tendo como entrada $10$, $100$, $1000$, $10000$ e $1000000$ pelo menos. Usando algoritmos de teoria da probabilidade que, testando um grande número de pares, o quociente acima deve ficar próximo de $\frac{6}{\pi^2}$; veja \[**Giblin 1993, p.16**\]. Como é que esse valor se compara com os resultados experimentais que você obteve?

Vamos utilizar a função de $\mbox{mmdc}$ que já implementamos. O gerador de números aleatórios poderá selecionar qualquer número interio.

In [ ]:
rand(Int32,1,2)

In [ ]:
function exercicio9(n)
    A = rand(Int32,n)
    B = rand(Int32,n)
    return sum([1 for i = 1:n if mdc(max(A[i],-A[i]),max(-B[i],B[i])) == 1])/n
end

In [ ]:
function exercicio9()
for i = 1:6
    print(string(10^i) * ": " * string(exercicio9(10^i)) * "\n")
end
print("inf: " * string(6/pi^2) *"\n")
end
@time exercicio9()

## Capítulo 2

 ## Exercício 11
 Seja $n$ um inteiro positivo e $d(n)$ o número de divisores positivos de $n$. Dizemos que $n$ é altamente composto se $d(m)<d(n)$ para todo $m<n$. Escreva um programa que, tendo como entrada um número inteiro $r$, determina todos os números altamente compostos menores que $r$. Use o programa para fazer uma lista dos números altamente compostos menores que $5000$. Observe as fatorações dos números altamente compostos da sua lista: que propriedades destes números você deduz da observação desta lista? Estes números foram estudados pelo matemático indiano autodidata Srinivasa Ramanujan.

In [ ]:
function primos_aux(n, lista, div)
    n = Int(n)
    if n == 1 
        return lista
    elseif div==n # n eh primo
          return push!(lista,n)
    elseif n % div == 0 # div divide n
        return primos_aux(n/div,push!(lista,div), div)
    elseif div^2>n
        return lista
    else
        return primos_aux(n, lista, div + 2)
    end
end

In [ ]:
function primos(n)
    lista = [] # sorry for not be trully functional D:
    while n % 2 == 0 # it hurts more in me than in you
        push!(lista, 2) # but, performance, right?
        n = Int(n/2)
    end
    return if (x = primos_aux(n,lista,3)) != [] x else [n] end
end

In [ ]:
function divisores_aux(esperando, resp)
    if size(esperando)[1] > 0
        return divisores_aux(
            esperando[1:end-1],
                vcat(resp,
                     filter(
                        x -> !(x in resp),
                        map(x -> x * esperando[end], resp)))) 
    else
        return resp
    end
end
    
function divisores(n)
    return sort(divisores_aux(primos(n), [1]))
end

In [ ]:
d(n) = size(divisores(n))[1]

In [ ]:
e_altamente_composto(r) = eac_aux(r,r-1)

function eac_aux(n,m)
    if m == 1
        true
    elseif d(m) < d(n)
        eac_aux(n,m-1)
    else
        false
    end
end

In [ ]:
e_altamente_composto(20)

In [ ]:
todos_altamente_compostos(r) = [i for i = 2:r if e_altamente_composto(i)]

In [ ]:
@time todos_altamente_compostos(5000)

##  Exercícico 12

Escreva um programa que impelemente o algoritmo de Fermat para achar dois fatores de um número inteiro positivo que seja menor que $2^{32}$. Este exercício é o primeiro de uma sequência que termina com o exercício 8 do capítulo 1.

In [ ]:
function fermat(n)
    x = floor(sqrt(n))
    if x^2 == n
        return (x,x)
    else
        x += 1
        y = sqrt(x^2 - n)
        while !isinteger(y)
            x += 1
            y = sqrt(x^2 - n)
            if x == (n+1)/2
                return string(n) * " é primo"
            end
        end
        return (Int(x+y),Int(x-y))
    end
end

In [ ]:
@time fermat(1342127)

 # Capítulo 3
 
 ## Exercício 10
 
 O objetivo deste exercício é a elaboração de um programa para determinar alguns valores (interios) de $x$ para os quais um polinômio quadrático $f(x)$ assume valores primos. O programa deve ter como entrada $f(x) = ax^2 + bx +c$. Estes números serão inteiros positivos ou negativos, mas podemos supor que $a>0$. O programa deve tabelar $f(n)$, para $n$ variando entre $0$ e $100$, e indicar quais destes valores são primos. Para fazer isso, será necessário executar o crivo de Erastótenes para achar os primos menores que $|f(0)|$ ou $|f(100|$ - dependendo de qual dos dois for maior. Observe que vai ser necessário limitar os valores de $a$, $b$ e $c$, e verificar se os valores de $f(x)$ não estão ultrapassando a precisão dos inteiros da linguagem de programação que está sendo utilizada. Se você não tomar estes cuidados o programa vai produzir resultados incorretos. Aplique seu programa para os polinômios 
 1. $f(x) = x^2 + 1$
 2. $f(x) = x^2 - 69x + 1231$
 3. $f(x) = 2x^2 - 199$
 4. $f(x) = 8x^2 - 530x + 7631$
 
 O polinômio (2) é uma variação de um famoso exemplo que L. Euler obteve em 1772.

In [2]:
function eras_aux(lista, iter)
    len = size(lista)[1]
    if  iter >= len
        return lista
    elseif lista[iter] == 0
        return eras_aux(lista,iter+2)
    else
        step = iter
        iter += step
        while iter <= len
            lista[iter] = 0
            iter += step
        end
        return eras_aux(lista,step+2)
    end
end

function erastotenes(n)
    lista = [if i%2==0 0 else 1 end for i=1:n]
    lista[1] = 0
    return [x[1] for x in enumerate(eras_aux(lista,3)) if x[2] == 1 || x[1] == 2]
end    

erastotenes (generic function with 1 method)

In [3]:
erastotenes(100)

25-element Array{Int64,1}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
 31
 37
 41
 43
 47
 53
 59
 61
 67
 71
 73
 79
 83
 89
 97

In [ ]:
function exercicio10(a,b,c)
    f(n) = a * n^2 + b * n + c
    primos = erastotenes(max(abs(f(0)),abs(f(100))))
    return [(n, f(n),if f(n) in primos "é primo" else "não primo" end) for n=1:100]
end

In [ ]:
@time exercicio10(1,0,1)

In [ ]:
@time exercicio10(1,-69,1231)

In [ ]:
@time exercicio10(2,0,-199)

In [ ]:
@time exercicio10(8,-530,7681)

 ## Exercício 11
 
 Vimos na seção 5 que existem várias formas que dão aproximações para $\pi(x)$, o número de primos positivos menores ou iguais a $x$. A fórmula decorrente do teorema dos números primos é $\frac{x}{\log x}$ que não fornece uma boa aproximação a não ser que o valor de $x$ seja enorme. Neste exercício desejamos fazer o estudo experimental de uma outra forma que serve de aproximação para $\pi(x)$. A fórmula é $$ S(x)=\frac{x}{\log x} \left(1+\left[\sum_{k=0}^{12} a_k\left(\log \log x\right)^k\right]^{-\frac{1}{4}}\right) $$
 onde $\log$ denota o *logaritmo natural* e $$a_0 = 229168.50747390,\ a_1 = -429449.7206839,\ a_2=199330.41355048,\ a_3=28226.22049280,\\ a_4 = 0,\ a_5=0,\ a_6 = -34712.81875914,\ a_7=0,\ a_8 = 33820.10886195,\\ a_9=-25379.82656589,\ a_{10}=8386.14942934,\ a_{11} = -1360.44512548,\ a_{12}=89.14545378. $$
 
Escreva um programa, baseado no *Crivo de Erastótenes* que, tendo como entrada o inteiro $x$, calcula $\pi(x)$. Use este algoritmo para gerar uma tabela com os valores de $\pi(x) - S(x)$ quando $x$ é igual a $11$, $100$, $1000$, $2000$, $3000$, $\dots$, $9000$ e $10000$. Comare com os valores correspondentes de $\pi(X) - \frac{x}{\log x}$. O que você conclui da análise destas tabelas?

In [ ]:
function S(x) 
    a = [229168.50747390,
        -429449.7206839,
        199330.41355048,
        28226.22049280,
        0,
        0,
        -34712.81875914,
        0,
        33820.10886195,
        -25379.82656589,
        8386.14942934,
        -1360.44512548,
        89.14545378]
    return (x/log(x)) * (1 + sum([a[k+1] * (log(log(x)))^k for k=0:12])^(-1/4))
end

In [ ]:
Π(x) = size(erastotenes(x))[1]

In [ ]:
defaultaprox(x) = x\log(x)

In [ ]:
function exercicio11()
    values = [11, 100, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    pilist = [Π(x) for x in values]
    slist = [S(x) for x in values]
    deflist = [defaultaprox(x) for x in values]
    PiSMinus = [pilist[i[1]]-slist[i[1]] for i in enumerate(values)]
    PiDefMinus = [pilist[i[1]]-deflist[i[1]] for i in enumerate(values)]
    DataFrame(value=values, pi_minus_s=PiSMinus, pi_minus_default = PiDefMinus)
end
@time exercicio11()

Eu concluo que a formula Default aumenta o erro conforme o valor aumenta, enquanto a fórmula $S$ mágica, mantem o erro estável.

## Exercício 12

Já vimos que um primo ímpar ou é da forma $4n+1$ ou da forma $4n+3$. Além disso, sabemos pelo exercício 7 que há infinitos primos da forma $4n+3$. Também há infinitos primos da forma $4n+1$, mas a demonstração é mais difícil. O objetivo deste exercício é estudar experimentalmente a frequência relativa de cada um destes dois tipos de primos. Seja $x$ um número real positivo. Denotaremos por $\pi_1(x)$ o número de primos positivos que são da forma $4n+1$; e por $\pi_3(x)$ o número correspondente de primos da forma $4n+3$. Escreva um programa, baseado no *Crivo de Erastótenes*, que, tendo como entrada um inteiro positivo $x$, determina $\pi_1(x)$ e $\pi_3(x)$. Tabele $\pi_1(x)$, $\pi_3(x)$ e $\frac{\pi_1(x)}{\pi_3(x)}$ para os valores de $100$ em $100$ entre $100$ e $10^5$. Sabe-se que $\lim_{x\rightarrow \infty} \frac{\pi_1(x)}{\pi_3(x)} = 1$. Os valores da sua tabela condizem com este resultado?

In [ ]:
function is_4n_mais_1(x)
    return (x-1)%4 == 0
end

function exercicio12(x) 
    primos = erastotenes(x)[2:end]  #queremos descartar o 2, o primeiro primo
    pi1 = [x for x in primos if is_4n_mais_1(x)]
    return (size(pi1)[1], size(primos)[1] - size(pi1)[1]) #retorna o pi1 e total de primos menos o pi1, que é o pi3
end

In [ ]:
exercicio12(100)

In [ ]:
function exercicio12()
    Π1 = []
    Π3 = []
    ΠDiv = []
    values = []
    for num = 100:100:10^5
        π1, π3 = exercicio12(num)
        push!(values,num)
        push!(Π1,π1)
        push!(Π3,π3)
        push!(ΠDiv,π1/π3)
    end
    showall(DataFrame(value = values, Π1 = Π1, Π3 = Π3, ΠDiv = ΠDiv))
end
@time exercicio12()

**Sim**, os valores condizem.

## Exercício 13

Adapte o programa elaborado no exercício anterior para determinar o menor valor inteiro de $x$ para o qual $\pi_1(x) > \pi_3(x)$. 

O exame da evidência experimental entõ disponível leou alguns matemáticos do começo do século \[sic: passado\] à conclusão de que, exceto para alguns valores relativamente pequenos de $x$, a desigualdade $\pi_1(x) < \pi_3(x)$ seria sempre verdadeira. A verdade emergiu em 1914, quando o matemático J. E. Littlewood mostrou que existem sequências infinitas $$x_1,x_2,\dots \ \mbox{e}\ y_1,y_2,\dots$$ de números reais positivos, tais que $$\lim_{i\rightarrow \infty} (\pi_1(x_i) -\pi_3(x_i)) = \infty \ \mbox{e}\ \lim_{i\rightarrow \infty} (\pi_1(y_i) -\pi_3(y_i)) = -\infty.$$

A moral é óbvia: é sempre muito perigoso generalizar a partir de dados numéricos, mesmo quando a quantidade de dados disponíveis é muito grande.

In [ ]:
function exercicio13()
    x = 1
    π1, π3 = exercicio12(x) #
    while π1 <= π3
        x += 1
        π1, π3 = exercicio12(x)
    end
    return x
end

In [ ]:
@time exercicio13()

# Capítulo 4

## Exercício 11

Escreva um programa que implemente o algoritmo para calcular potências módulo $n$ que é descrito na seção 2 do apêndice. O algoritmo deverá ter como entrada $a$, $k$ e $n$, onde $a$ é um inteiro qualquer $k$ e $n$ são inteiros positivos. A saída deverá ser a forma reduzida de $a^k$ módulo $n$. Este algoritmo é uma parte fundamental de quase todos os algoritmos que estudaremos a partir do capítulo 6.

### Algoritmo de exponenciação:

**Entrada:** *Inteiros $a$, $e$ e $n$, $n>0$ e $e\geq 0$*
**Saída:**  *a forma reduzida de $a^e$ módulo $n$.*

**Etapa 1:** *Comece com $A=a$, $P=1$ e $E=e$.* 

**Etapa 2:** *Se $E=0$ imprime '$a^e\equiv P\ (\mbox{mod}\ n)$'; se não vai para Etapa 3;*

**Etapa 3:** *Se $E$ for ímpar então atribua a $P$ o valor do resto da divisão de $A\cdot P$ por $n$ e a $E$ o valor $\frac{E-1}{2}$ e vá para a Etapa 5, se não vá para a Etapa 4;*

**Etapa 4:** *Se $E$ for par então atribua a $E$ o valor $\frac{E}{2}$ e vá para a Etapa 5.*

**Etapa 5:** *Substitua o valor atual de $A$ pelo resto da divisão de $A^2$ por $n$ e vá para a Etapa 2.*


In [ ]:
function exercicio11(a,e,n) ##feito puramente funcional :P
    return verifica_pos(etapa2(a,1,e,n),n)
end

function etapa2(A,P,E,n)
    if E==0
        return P
    else
        return etapa3(A,P,E,n)
    end
end   

function etapa3(A,P,E,n)
    if E%2 == 1
        return etapa5(A,(A * P) % n,(E-1)/2,n)
    else
        return etapa4(A,P,E,n)
    end
end

function etapa4(A,P,E,n)
    if E%2 == 0  ##eu acho este passo muito estranho, mas o pseudo-código do livro pediu pra fazer assim e assim o fiz
        return etapa5(A,P,E/2,n)
    end
end

function etapa5(A,P,E,n)
    return etapa2(A^2 % n, P, E, n)
end

function verifica_pos(a,n) #função que garante que apenas iremos retornar resultados positivos
    if a>=0
        return a % n
    else #se for negativo
        return (n+a) % n #retorna o correspondente positivo
    end
end

In [ ]:
exercicio11(10,135,7) ##na pagina 79 do livro temos que esse valor é 6

In [ ]:
exercicio11(5,3,13) ##retirado da Wikipedia, espera-se 8

In [ ]:
@time exercicio11(4,13,497) ##tambem retirado da Wikipedia, espera-se 445

# Capítulo 5

## Exercício 16

Use o exercício 14 para escrever um programa que, tendo como entrada $a$ e $p$, calcula o inverso de $a$ módulo $p$. Por segurança faça seu programa verificar primeiro que $p$ *realmente* não divide $a$. Use isto para escrever um outro programa que determine a solução de $ax\equiv b\ (\mbox{mod}\ p)$, dada as entradas $a$, $b$ e $p$.

#### Exercício 14

Seja $p$ um número primo e $a$ um inteiro que não é divisível por $p$. Mostre que o invero de $\bar{a}$ em $\mathbb{Z}_p$ é $\bar{a}^{p-2}$.

In [ ]:
function inverso(a,p)
    if p in erastotenes(p) && a % p != 0  ##verifica se p é primo e se p não divide a
        return ((a % p)^(p-2) % p)
    else
        throw(ArgumentError)
    end
end

In [ ]:
inverso(2,5) #esperamos que seja 3

Note que $x\equiv ba^{-1}\ (\mbox{mod}\ p)$. Isso é tudo que precisamos

In [ ]:
exercicio16(a,b,p) = (b * inverso(a,p)) % p

Vamos testar nosso programa para $8x \equiv 3\ (\mbox{mod}\ 7)$. Esperamos que $x = 3$, porque $8\equiv 1\ (\mbox{mod}\ 7)$.

In [ ]:
@time exercicio16(8,3,7)

# Exercício 17

Seja $p = 4k+3$ um primo positivo. Escreva um programa que, tendo por entrada $p$ e um inteiro positivo $a$, calcula as duas soluções de $x^2\equiv a\ (\mbox{mod}\ p)$. Observe que sabemos do exercício 15 que *se esta equação tem solução* $b$ então $b \equiv a^{k+1}\ (\mbox{mod}\ p)$. Assim o programa deve calcular a forma reduzida de $a^{k+1}$ módulo $p$ e, em seguida, verificar se, de fato, esta é uma solução da equação dada. A saída será constituída pelas soluções da equação, ou por uma mensagem indicando que a equação dada não tem solução. Este exercício é o segundo de uma sequência que termna com o exercício 8 do capítulo 11.

In [ ]:
function extrai_k(p)
    if p in erastotenes(p) && p % 4 == 3 #se é primo e de fato da forma 4k+3
        return (p-3)/4
    else
        throw(ArgumentError)
    end
end

function exercicio17(a,p)
    b = a^(extrai_k(p)+1) % p
    if b^2 % p == a
        return (b,-b)
    else
        return "Não tem solução"
    end
end

Vamos testar com $x^2 \equiv 9\ (\mbox{mod}\ 11)$ que sabemos que tem que ser igual a $\pm 3$.

In [ ]:
@time exercicio17(9,11)

## Exercício 18

Vários problemas da Teoria dos Números (como o Último Teorema de Fermat) requerem primos $p$ que satisfaçam a congruência $$a^{p-1} \equiv 1\ (\mbox{mod}\ p^2)$$ para algum inteiro $a$. Escreva um programa que, tendo por entrada dois inteiros positivos $a>1$ e $r$, determina os primos $p$ que satisfaçam a congruência acima e tais que $a+1\leq p \leq r$. Observe que o programa terá que aplicar o crivo de Eratóstenes para achar os primos menos ou iguais a $r$. Estes primos serão então testados na congruência acima. Se $r=10^5$ então o número de primos satisfazendo a congruência é dois quando $a=2,\ 5,\ 10$ e $14$; e cinco quando $a=19$, por exemplo.

In [ ]:
## também feito puramente funcional :P
## a gente filta o erastotenes com uma expressão lambda (x) -> a+1 <= x
## também usamos a função exercício11 que serve para realizar a exponenciação de a^e (mod n).
exercicio18(a,r) = [p for p in filter((x) -> a+1 <= x , erastotenes(r)) if exercicio11(a,p-1,p^2) == 1] 

In [ ]:
@time exercicio18(5,10^5)

In [ ]:
@time exercicio18(10, 10^5) #TODO nao sei porque não funciona

In [ ]:
@time exercicio18(14, 10^5)

In [ ]:
@time exercicio18(19,10^5) ##TODO não sei porque não funciona

Eu implementei em Python também, usando um Crivo de Eratóstenes que peguei no Rosetta Code (tentei adaptar o que escrevi aqui, mas o Python tilta na recursão) e usando a função pow do Python. Deu os mesmos resultados. Então não sei o que está pegando.

# Capitulo 6

## Exercício 8

Escreva um programa que, tendo como entrada um inteiro positivo $r$, determina todos os números entre $1$ e $r$ que sao pseudoprimos para as bases $2$ e $3$. Lembre-se que tal pseudoprimo $n$ é um número *ímpar* e *composto* que satisfaz as equações $$2^{n-1}\equiv 1\ (\mbox{mod}\ n)\ \mbox{e}\ 3^{n-1}\equiv 1\ (\mbox{mod}\ n).$$

Portanto o programa só precisa testar os números ímpares e além destas congruências, tem que verificar se os números obtidos são primos ou não. Uma maneira de fazer isso é programar o Crivo de Eratóstenes de modo a conservar a lista dos ímpares compostos, em vez dos primos e depois testar as congruências. Aplique o programa  com $r=10^5$. Quantos e quais foram os pseudoprimos obtidos? Quantos destes são números de Carmichael?

In [ ]:
function e_pseudoprimo(num,base, primos)
    if !(num in primos && num % 2 == 1)
        return exercicio11(base,num-1,num)==1
    else
        return false
    end
end

In [ ]:
function e_carmichael(n)
    if n in [1,2,3]
        return false
    end
    if n%2 == 1 && !(n in erastotenes(n))
        return reduce((x,y)-> x && y, [exercicio11(b,n,n) == b for b=2:(n-2)])
    else
        return false
    end
end

In [ ]:
e_carmichael(559)

In [ ]:
[num for num=1:600 if e_carmichael(num)] ##todos números carmichael até 600

In [ ]:
function exercicio8(r) 
    primos = erastotenes(r)
    return [n for n=1:2:r if e_pseudoprimo(n,2,primos) && e_pseudoprimo(n,3,primos)]
end

In [ ]:
@time lista = exercicio8(10^5)

In [ ]:
print("Foram " * string(size(lista)[1]) * " pseudoprimos obtidos")

In [ ]:
print("Temos " * string(size(filter((x)->e_carmichael(x),lista))[1]) * " números de Carmichael")

## Exercício 9

O objetivo deste exercício é a elaboração de um programa para determinar todos os números de Carmichael que são produtos de $d$ primos, todos menores que $10^3$. O principal problema é que, mesmo para valores relativamente pequenos de de $d$, alguns dos números de Carmichael obtidos podem ser bem grandes. Isto torna difícil verificar que $p-1$ divide $n-1$, quando $p$ é um fator primo de $n$. Mas é fácil contornar este problema usando congruências. Digamos que o programa gerou o número $n = p_1p_2\dots p_d$. Precisamos verificar se $n-1$ é ou não divisível por $p_1-1$, por exemplo. Como $n$ aparece fatorado, você pode multiplicar os fatores um de cada vez e ir reduzindo módulo $p_1-1$ a cada produto, de modo a manter o resultado pequeno. Como os fatores são menores que $10^3$ isto vai ser muito rápido de executar. Aplique o seu programa para $d=1,\dots,8$. Quais números de Carmichael obtidos em cada caso? Não é necessário multiplicar os fatores para obter o número, basta que o programa liste os fatores.

In [34]:
function is_carmichael(primefaclist, counter = 1)
    if counter == size(primefaclist)[1] + 1
        return true
    else
        if ((reduce((x,y)->(x*y) % (primefaclist[counter]-1),primefaclist) - 1) % (primefaclist[counter]-1)) != 0
            return false
        else
            return is_carmichael(primefaclist,counter+1)
        end
    end
end

,
function carmichael_d_primos(d, primos)
    if d == 1 || d == 2
        return []
    else
        return [n for n in Combinatorics.Combinations(primos[2:end],d) if is_carmichael(n)]
    end
end

carmichael_d_primos (generic function with 1 method)

In [10]:
function cap6exercicio9()
    prims = erastotenes(10^3)
    ANS = [carmichael_d_primos(d,prims[2:end]) for d in 1:8]
    for x in enumerate(ANS)
        print("Números d")
        print(x)
    end
end
#@time cap6exercicio9()

cap6exercicio9 (generic function with 1 method)

In [35]:
function list_carmichael(primos, prev=[], prev_p1=[])
    if size(prev)[1] >= 8
        return prev
    else
        if is_carmichael(prev)
            ANS = []
            for primo in primos
                #part1 verificar se para todo prev_p1 ele é igual a 0
                ((reduce((x,y)->(x*y) % (primefaclist[counter]-1),primefaclist) - 1) % (primefaclist[counter]-1)) != 0
                for p_1 in prev_p1
                    if p_1*primo % (prev)
        else
            return aux()
    end
end

true

In [46]:
A = [1,2,3]
B = [[[1,2],[2,3]],[[4,5,6],[7,8]]]
reduce((a,b)->vcat(a,b),B)

4-element Array{Array{Int64,1},1}:
 [1, 2]   
 [2, 3]   
 [4, 5, 6]
 [7, 8]   

In [45]:
prims = erastotenes(10^3)

168-element Array{Int64,1}:
   2
   3
   5
   7
  11
  13
  17
  19
  23
  29
  31
  37
  41
   ⋮
 919
 929
 937
 941
 947
 953
 967
 971
 977
 983
 991
 997

In [27]:
primos8 = Combinatorics.Combinations(prims[2:end],7)

Combinatorics.Combinations{Array{Int64,1}}([3, 5, 7, 11, 13, 17, 19, 23, 29, 31  …  937, 941, 947, 953, 967, 971, 977, 983, 991, 997], 7)

In [32]:
function teste(primos8)
    counter = 0
    ANS = []
    for primos in primos8
        push!(ANS,is_carmichael(primos))
        counter += 1
        if counter == 40000000
            break
        end
    end
    return ANS
end

teste (generic function with 1 method)

In [33]:
@time teste(primos8)

 16.972550 seconds (300.01 M allocations: 16.693 GiB, 15.67% gc time)


40000000-element Array{Any,1}:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [ ]:
#IDEIA PARA RESOLVER
#Passo uma árvore de números primos

#Se descobrimos que algum nó da árvore não é de Carmichael, não testamos os nós abaixo dali


In [11]:
@time cap6exercicio9()

InterruptException: InterruptException:

In [ ]:
comb = [[x for x in Combinatorics.Combinations(erastotenes(10^3),parts)] for parts=2:8]

In [ ]:
reduce(+,map((x)->size(x)[1],comb))

## Exercício 10

Escreva um programa para determinar o menor pseudoprimo forte para uma dada base. Vai ser necessário implementar o teste de Miller, de modo que a entrada seja um inteiro positivo $b>1$. O programa deve aplicar o teste de Miller na base $b$ aos ímpares compostos, até achar o primeiro número para qual o teste é inconclusivo. Uma maneira de se fazer isso, é programar o Crivo de Eratóstenes de maneira a conservar a lista dos ímpares compostos, em vez dos primos e depois testar as congruências. Aplique o teste para as bases $2$, $3$, $5$ e $7$. Quais os resultados obtidos?

In [ ]:
using Distributed
@parallel for N in 1:5:20
    println("The N of this iteration in $N")
end

In [ ]:
1+1